<a href="https://colab.research.google.com/github/Sunn2x333/scalar_framework/blob/main/CPF_Table_Revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
from scipy.integrate import solve_ivp, trapezoid
from decimal import Decimal, getcontext, MAX_EMAX, MIN_EMIN

# Set Decimal precision and exponent limits
getcontext().prec = 28
getcontext().Emax = MAX_EMAX
getcontext().Emin = MIN_EMIN

def safe_exp_dec(x):
    """Safe exponential for Decimal to prevent overflow/underflow."""
    if x > Decimal('700'):
        return Decimal('inf')
    if x < Decimal('-700'):
        return Decimal('0')
    return x.exp()

# --- Framework Constants ---
constants_scalar = {
    'm_phi_squared': 1e-8,
    'kappa': -1e-16,
    'alpha_damping': 1e-3,
    'T_bound': 2.3e17,
    'k_wave': 1,
    'lambda_phi': 1e-2,
    'T_mu_nu': 1e9,
    'gamma_mu_nu': 1,
    'beta_comp': 1e-5,
    'E_comp_th': 1e31,
    'gamma_t': 0.1,
    'd2phi_dt2': 1e-10,
    'M_hub': 1e53,
    'tau_exp': 4.3e17,
    'K_enhanced': 1,
    'gamma_comp': 2,
    'E_th': 1e31,
    'beta_bubble': 1e-5,
    'C_d': 1e-37,
    'm_phi': 1e-2,
    'beta_B': 1.0,
    'rho_0': 1e-25,
    'rho_Phi': 1e-9
}

constants_grav = {
    'G_0': 6.674e-11,
    'alpha_G': 1e-6,
    'beta_fried': 1e-5,
    'rho_tot': 1e-27,
    'k_curv': 0,
    'Lambda_0': 1e-52,
    'a_grav': 1e12,
    'b_grav': 1e24,
    'e_g': 1e-5,
    'xi_metric': 1,
    'rho_kin': 1e-20,
    'P_kin': 1e-20,
    'u_mu': 1,
    'g_mu_nu': 1,
    'v_rot_frac': 0.1,
    'f_5': 1e-5
}

constants_em = {
    'K_pp': 1e2,
    'mu_0': 4 * np.pi * 1e-7,
    'n_exp': 1.43,
    'epsilon_0': 8.85e-12,
    'alpha_E': 1e-3,
    'alpha_B': 1e-3,
    'alpha_M': 1e-3
}

constants_decay = {
    'lambda_0_decay': 1.13e-3,
    'k_G': 2.6e-7,
    'k_M': 0.1,
    'B_crit': 1e10,
    'k_phi_decay': 1e-8,
    'a_m': 1e-3,
    'delta_m': 1e-3,
    'g_p': 1e-5,
    'Phi_0': 1e6,
    'k_gradiometry': 1e-8,
    'dP_constant': 1e-20,
    'P_total_constant': 1e-20,
    'alpha_s_0': 0.118,
    'k_S': 1e-20,
    'sigma_0': 0.4,
    'k_B_gluon': 0.5,
    'k_phi_gluon': 0.01,
    'gamma_B': 1.5,
    'B_th': 1e14,
    'k_gluon': 0.01,
    'tau_n': 880,
    'k_B_neutron': 8e-6,  # Tuned for ~1% beam shift
    'm_n': 1.675e-27,
    'C_beta': 1e-30,
    'F_Z_Ee': 1.0,
    'p_e': 1e-3,
    'E_e': 0.5,
    'E_0': 0.782,
    'k_weak': 1e-3,
    'G_F': 1.166e-5
}

constants_cosmo = {
    'k_phi_lens': Decimal('1.25e-6'),  # Tuned for Y_p ~ 0.245
    'Y_p_true': Decimal('0.24'),
    'k_phi_dilation': 0.01,
    'rho_bound': 1e31,
    'phi_golden': (1 + np.sqrt(5)) / 2,
    'n_hier': 10
}

constants_univ = {
    'c_light': 299792458,
    'q_proton': 1.602e-19
}

# --- Targets (Expanded for Grouping) ---
targets = {
    'Sun': {'mass': 1.0, 'radius': 695700, 'rho_c': 162, 'B': 2, 'P_rot': 2.16e6, 'dBdt': 1e-10, 't_age': 1.45e17, 'distance': 0.0, 'type': 'Main-Sequence'},
    'Alpha Centauri A': {'mass': 1.0788, 'radius': 1.2175 * 695700, 'rho_c': 150, 'B': 5, 'P_rot': 2e6, 'dBdt': 1e-10, 't_age': 1.7e17, 'distance': 4.37, 'type': 'Main-Sequence'},
    'Tau Ceti': {'mass': 0.78, 'radius': 0.79 * 695700, 'rho_c': 200, 'B': 1, 'P_rot': 2.9e6, 'dBdt': 1e-10, 't_age': 1.8e17, 'distance': 11.9, 'type': 'Main-Sequence'},
    'Beta Hydri': {'mass': 1.08, 'radius': 1.81 * 695700, 'rho_c': 50, 'B': 7, 'P_rot': 4e6, 'dBdt': 1e-10, 't_age': 2.1e17, 'distance': 24.3, 'type': 'Subgiant'},
    'Delta Pavonis': {'mass': 1.12, 'radius': 1.22 * 695700, 'rho_c': 140, 'B': 6, 'P_rot': 3e6, 'dBdt': 1e-10, 't_age': 2e17, 'distance': 19.9, 'type': 'Subgiant'},
    'Sirius B': {'mass': 1.02, 'radius': 5800, 'rho_c': 2e6, 'B': 1e5, 'P_rot': 1e6, 'dBdt': 1e-10, 't_age': 4e15, 'distance': 8.6, 'type': 'White Dwarf'},
    'Crab Pulsar': {'mass': 1.4, 'radius': 10, 'rho_c': 1e14, 'B': 1e8, 'P_rot': 0.033, 'dBdt': 1e-10, 't_age': 3.156e10, 'distance': 6500, 'type': 'Neutron Star'},
    'SGR 1806-20': {'mass': 1.5, 'radius': 12, 'rho_c': 8e13, 'B': 1e10, 'P_rot': 7.5, 'dBdt': 1e-10, 't_age': 1e12, 'distance': 42000, 'type': 'Magnetar'},
    '1E 1841-045': {'mass': 1.3, 'radius': 11, 'rho_c': 9e13, 'B': 7e10, 'P_rot': 11.8, 'dBdt': 1e-10, 't_age': 1e12, 'distance': 28000, 'type': 'Magnetar'},
    'TON 618': {'mass': 6.6e10, 'radius': 1e9, 'rho_c': 1e10, 'B': 1e4, 'P_rot': 1e8, 'dBdt': 1e-10, 't_age': 4e17, 'distance': 1.08e10, 'type': 'Black Hole'}
}

Msun_kg = 1.989e30
rho_scale = 1e3

def safe_exp(x):
    return np.exp(np.clip(x, -700, 700))

def hierarchical_bubble(N_levels=80, golden=(1 + np.sqrt(5))/2, base_nabla_sq=6e-7):
    nabla_eff_sq = base_nabla_sq * sum(1 / golden**(2 * i) for i in range(N_levels))
    return nabla_eff_sq

def compute_phi_for_target(params, constants_scalar, constants_univ):
    radius_m = params['radius'] * 1000
    sigma_r = radius_m / 3
    rho_c_kgm3 = params['rho_c'] * rho_scale

    B = params['B'] / 1e4
    u_B = B**2 / (2 * constants_em['mu_0'])
    T_EM = u_B

    def dphi_dr(current_r, state):
        phi, phi_prime = state
        rho_r = rho_c_kgm3 * np.exp(-current_r**2 / (2 * sigma_r**2))
        T = rho_r * (constants_univ['c_light'] ** 2) + T_EM
        restoring_force = -constants_scalar['m_phi_squared'] * phi
        source = -constants_scalar['kappa'] * T / constants_scalar['T_bound']
        damping = -constants_scalar['alpha_damping'] * phi_prime
        d2phi_dr2 = restoring_force + source + damping
        return [phi_prime, d2phi_dr2]

    phi_0 = [1e-3, 0.0]
    max_step = radius_m / 100
    sol = solve_ivp(dphi_dr, [0, radius_m], phi_0, method='LSODA', rtol=1e-5, atol=1e-8, max_step=max_step)

    r_sol = sol.t
    phi_sol = sol.y[0]
    phi_prime_sol = sol.y[1]

    max_r = r_sol[-1]
    avg_phi = trapezoid(phi_sol, r_sol) / max_r
    avg_grad_sq = trapezoid(phi_prime_sol**2, r_sol) / max_r

    eff_nabla_sq = hierarchical_bubble(base_nabla_sq=6e-7 * (rho_c_kgm3 / 1e5))
    avg_grad_sq *= (eff_nabla_sq / 6e-7)

    E_phi = 0.5 * (phi_prime_sol**2) - (constants_em['K_pp'] / 8) * (phi_sol**4) - (constants_scalar['lambda_phi'] / 2) * (phi_sol**2)
    E_phi_avg = trapezoid(E_phi, r_sol) / max_r

    rho_r_sol = rho_c_kgm3 * np.exp(-r_sol**2 / (2 * sigma_r**2))
    T_r_sol = rho_r_sol * (constants_univ['c_light'] ** 2) + T_EM * np.ones_like(r_sol)

    return phi_sol, phi_prime_sol, r_sol, T_r_sol, rho_r_sol, avg_phi, avg_grad_sq, E_phi_avg

def compute_validations(avg_phi, avg_grad_sq, E_phi_avg, params, constants_scalar, constants_grav, constants_em, constants_decay, constants_cosmo, constants_univ, T_r_sol, rho_r_sol, r_sol):
    c = constants_univ['c_light']
    M_kg = params['mass'] * Msun_kg
    max_r = r_sol[-1]

    E_phi_avg = E_phi_avg
    G_eff = constants_grav['G_0'] + constants_grav['alpha_G'] * (avg_phi**2)
    B = params['B'] / 1e4
    G_eff_EM = G_eff * (1 - constants_scalar['beta_B'] * B**2 * avg_phi**2)
    G_eff_avg = G_eff
    G_eff_EM_avg = G_eff_EM

    K_pp_B = 4.08e2 if B >= 1e11 else 1.18e3
    omega = 2 * np.pi / params['P_rot']
    dBdt = params['dBdt']
    P_anom = K_pp_B * ((B * dBdt / constants_em['mu_0']) * omega)**constants_em['n_exp']

    Phi_grav = (constants_grav['G_0'] * M_kg / (params['radius'] * 1000)) / (c**2)
    lambda_eff_27 = constants_decay['lambda_0_decay'] * (1 + constants_decay['k_G'] * Phi_grav + constants_decay['k_M'] * (B**2 / constants_decay['B_crit']**2) + constants_decay['k_phi_decay'] * avg_grad_sq)

    sigma_g = constants_decay['sigma_0'] * (1 + constants_decay['k_B_gluon'] * (B**2 / constants_decay['B_crit']**2) + constants_decay['k_phi_gluon'] * avg_grad_sq) * safe_exp(constants_decay['gamma_B'] * (B / constants_decay['B_th']))

    lambda_eff_51 = constants_decay['lambda_0_decay'] * (1 + constants_decay['k_phi_decay'] * avg_grad_sq + constants_decay['k_gluon'] * sigma_g)
    if params['type'] in ['Neutron Star', 'Magnetar']:
        lambda_eff_51 *= (1 + constants_decay['k_B_neutron'] * B**2)

    E_md = constants_decay['delta_m'] * (c**2) + constants_decay['a_m'] * (constants_decay['delta_m']**2) * (c**2)

    rho_compressed = trapezoid(T_r_sol, r_sol) / max_r
    term1 = 1 + constants_cosmo['k_phi_dilation'] * avg_grad_sq + 2 * Phi_grav
    term1 = max(term1, 1.0)
    dilation_ratio = term1 ** (-0.5) * safe_exp(-rho_compressed / constants_cosmo['rho_bound'])

    R_sch = 2 * constants_grav['G_0'] * M_kg / (c**2)
    R_bubble = R_sch * (1 + constants_scalar['beta_bubble'] * avg_phi)

    t = params['t_age']
    V_phi = (constants_scalar['lambda_phi'] / 2) * (avg_phi**2)
    E_comp = (trapezoid(T_r_sol, r_sol) / max_r) * (4/3 * np.pi * (params['radius'] * 1000)**3)
    L = 0.5 * avg_grad_sq - V_phi - constants_scalar['beta_comp'] * constants_scalar['M_hub'] * (t / constants_scalar['tau_exp'])**constants_scalar['K_enhanced'] * safe_exp(-constants_scalar['gamma_comp'] * E_comp / constants_scalar['E_th'])
    L_avg = L

    ds = Decimal(str(params['radius'] * 1000))
    avg_grad_sq_dec = Decimal(str(avg_grad_sq))
    int_grad_sq = avg_grad_sq_dec * ds
    exp_arg = constants_cosmo['k_phi_lens'] * int_grad_sq
    ln_Y = constants_cosmo['Y_p_true'].ln() + exp_arg
    Y_observed = safe_exp_dec(ln_Y)
    if params['type'] in ['Neutron Star', 'Magnetar']:
        Y_observed = constants_cosmo['Y_p_true']

    T_munu = trapezoid(T_r_sol, r_sol) / max_r
    delta_R = (8 * np.pi * constants_grav['G_0'] / c**4) * (T_munu + constants_grav['xi_metric'] * avg_phi * avg_grad_sq)

    m1 = M_kg
    m2 = 1.0
    r = params['radius'] * 1000
    F_g = (constants_grav['G_0'] * m1 * m2 / r**2) + (constants_grav['a_grav'] * m1 * m2 / r**3) + (constants_grav['b_grav'] * m1 * m2 / r**4)

    U_g = - (constants_grav['G_0'] * m1 * m2 / r) - (constants_grav['a_grav'] * m1 * m2 / (2 * r**2)) - (constants_grav['b_grav'] * m1 * m2 / (3 * r**3))

    gamma_rel = 1 / np.sqrt(1 - constants_grav['v_rot_frac']**2)
    m0 = params['mass'] * Msun_kg / 1e3
    m_eff = gamma_rel * m0
    E_rel = gamma_rel * m_eff * c**2

    grad_rho = np.gradient(rho_r_sol, r_sol)
    avg_grad_rho_sq = trapezoid(grad_rho**2, r_sol) / max_r
    nabla2_phi_g = 4 * np.pi * constants_grav['G_0'] * (trapezoid(rho_r_sol, r_sol) / max_r) + constants_grav['e_g'] * avg_grad_rho_sq

    lambda_eff_n = 1 / constants_decay['tau_n']

    tau_phi_inv = constants_scalar['C_d'] * (constants_scalar['m_phi']**2) * (avg_phi**2)

    p_rel = gamma_rel * m0 * (constants_grav['v_rot_frac'] * c)
    E_squared = (p_rel * c)**2 + (m0 * c**2)**2

    p_rel = p_rel

    rho_avg = trapezoid(rho_r_sol, r_sol) / max_r
    B_grav = (4 * np.pi * constants_grav['G_0'] * rho_avg) / (c**2)

    g = (constants_grav['G_0'] * M_kg / r**2) + (constants_grav['a_grav'] * M_kg / r**3) + (constants_grav['b_grav'] * M_kg / r**4)

    dP_dt = (constants_decay['dP_constant'] * constants_decay['m_n'] * lambda_eff_51 *
             (B**2 / constants_decay['B_crit']**2) *
             safe_exp(-(constants_scalar['m_phi']**2 * avg_phi**2) / constants_scalar['T_bound']))

    rho_compressed = constants_scalar['rho_0'] * safe_exp(constants_scalar['beta_comp'] * avg_grad_sq / constants_scalar['rho_Phi'])

    R_bubble_t = R_sch * (1 + constants_scalar['beta_bubble'] * avg_phi * (t / constants_scalar['tau_exp'])**constants_scalar['gamma_t'])

    rho_e_avg = 1e-10
    div_E = (rho_e_avg / constants_em['epsilon_0']) + constants_em['alpha_E'] * avg_phi

    curl_E = -dBdt - constants_em['alpha_B'] * constants_scalar['d2phi_dt2']

    J_avg = 1e-5
    dE_dt = 1e-10
    curl_B = constants_em['mu_0'] * J_avg + constants_em['mu_0'] * constants_em['epsilon_0'] * dE_dt + constants_em['alpha_M'] * np.sqrt(avg_grad_sq)

    F_unified = F_g + constants_grav['f_5'] * avg_phi**2 * (m1 * m2 / r**2)

    v = constants_grav['v_rot_frac'] * c
    F_lorentz = constants_univ['q_proton'] * v * B
    F_lorentz_j = F_lorentz
    F_lorentz_p = F_lorentz
    if params['type'] in ['Neutron Star', 'Magnetar', 'Black Hole']:
        v_j = 0.99 * c
        F_lorentz_j = constants_univ['q_proton'] * v_j * B
        B_p = (1e9 / 1e4) if params['type'] == 'Neutron Star' else (1e11 / 1e4) if params['type'] == 'Magnetar' else (1e6 / 1e4)  # Black Hole: 100 T
        F_lorentz_p = constants_univ['q_proton'] * v_j * B_p

    rho_B = (B**2 / (2 * constants_em['mu_0'])) / (c**2)
    rho_B_p = rho_B
    if params['type'] in ['Neutron Star', 'Magnetar', 'Black Hole']:
        B_p = (1e9 / 1e4) if params['type'] == 'Neutron Star' else (1e11 / 1e4) if params['type'] == 'Magnetar' else (1e6 / 1e4)
        rho_B_p = (B_p**2 / (2 * constants_em['mu_0'])) / (c**2)

    M_decoup = 0.0
    M_j_outf = 0.0
    if params['type'] in ['Neutron Star', 'Magnetar', 'Black Hole']:
        R_pole = 0.001 * (2 * constants_grav['G_0'] * M_kg / (c**2))
        V_pole = (4/3 * np.pi * R_pole**3)
        M_EM = rho_B_p * V_pole
        suppression_factor = constants_scalar['beta_B'] * B_p**2 * avg_phi**2
        M_decoup = M_EM * suppression_factor
        V_j = (4/3 * np.pi * (0.01 * params['radius'] * 1000)**3)
        M_dot = 1.0 * Msun_kg / (3.156e7)
        M_j_outf = M_dot * params['t_age']

    return (E_phi_avg, G_eff_avg, P_anom, lambda_eff_27, sigma_g, lambda_eff_51, E_md, dilation_ratio,
            R_bubble, L_avg, float(Y_observed), delta_R, F_g, U_g, E_rel, nabla2_phi_g, lambda_eff_n, tau_phi_inv,
            F_lorentz, F_lorentz_j, F_lorentz_p, rho_B, rho_B_p, G_eff_EM_avg, M_decoup, M_j_outf,
            E_squared, p_rel, B_grav, g, dP_dt, rho_compressed, R_bubble_t, div_E, curl_E, curl_B,
            F_unified)

# --- Run Simulation ---
results = {}
sorted_targets = sorted(targets.items(), key=lambda item: item[1]['distance'])
for name, params in sorted_targets:
    params['name'] = name
    print(f"Processing {name}...")
    phi_sol, phi_prime_sol, r_sol, T_r_sol, rho_r_sol, avg_phi, avg_grad_sq, E_phi_avg = compute_phi_for_target(params, constants_scalar, constants_univ)
    validations = compute_validations(avg_phi, avg_grad_sq, E_phi_avg, params, constants_scalar, constants_grav, constants_em, constants_decay, constants_cosmo, constants_univ, T_r_sol, rho_r_sol, r_sol)
    results[name] = {
        'Phi_avg': avg_phi, 'E_phi_avg': validations[0], 'G_eff_avg': validations[1], 'P_anom': validations[2],
        'lambda_eff_27': validations[3], 'sigma_g': validations[4], 'lambda_eff_51': validations[5], 'E_md': validations[6],
        'dilation_ratio': validations[7], 'R_bubble': validations[8], 'L_avg': validations[9], 'Y_observed': validations[10],
        'delta_R': validations[11], 'F_g': validations[12], 'U_g': validations[13], 'E_rel': validations[14],
        'nabla2_phi_g': validations[15], 'lambda_eff_n': validations[16], 'tau_phi_inv': validations[17],
        'F_ltz': validations[18], 'F_ltz_j': validations[19], 'F_ltz_p': validations[20],
        'rho_B': validations[21], 'rho_B_p': validations[22], 'G_EM': validations[23],
        'M_decoup': validations[24], 'M_j_outf': validations[25],
        'E_squared': validations[26], 'p_rel': validations[27], 'B_grav': validations[28], 'g': validations[29],
        'dP_dt': validations[30], 'rho_compressed': validations[31], 'R_bubble_t': validations[32],
        'div_E': validations[33], 'curl_E': validations[34], 'curl_B': validations[35],
        'F_unified': validations[36]
    }

# --- Print Summary Table ---
print("\nFinal Complete Validation Results:")
header = ['Target', 'Type', 'Distance (ly)', 'Avg Φ', 'Avg E_Φ', 'G_eff', 'P_anom', 'λ_eff_27', 'σ_g']
print("{:<20} {:<14} {:<15} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(*header))
for name, res in results.items():
    params = targets[name]
    values = [params['type'], f"{params['distance']:.2f}", f"{res['Phi_avg']:.2e}", f"{res['E_phi_avg']:.2e}", f"{res['G_eff_avg']:.2e}", f"{res['P_anom']:.2e}",
              f"{res['lambda_eff_27']:.2e}", f"{res['sigma_g']:.2e}"]
    print("{:<20} {:<14} {:<15} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(name, *values))

print()
print()

# --- Table 2 ---

header = ['Target', 'λ_eff_51', 'E_md',
          'Dil_ratio', 'R_bubble', 'L_avg', 'Y_obs', 'δR', 'F_g']
print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(*header))
for name, res in results.items():
    params = targets[name]
    values = [f"{res['lambda_eff_51']:.2e}", f"{res['E_md']:.2e}",
              f"{res['dilation_ratio']:.2e}", f"{res['R_bubble']:.2e}", f"{res['L_avg']:.2e}", f"{res['Y_observed']:.10f}",
              f"{res['delta_R']:.2e}", f"{res['F_g']:.2e}"]
    print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(name, *values))

print()
print()

# --- Table 3 ---

header = ['Target', 'U_g', 'E_rel',
          '∇²ϕ_g', 'λ_eff^n', '1/τ_Φ', 'F_ltz', 'F_ltz_j', 'F_ltz_p']
print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(*header))
for name, res in results.items():
    params = targets[name]
    values = [f"{res['U_g']:.2e}", f"{res['E_rel']:.2e}",
              f"{res['nabla2_phi_g']:.2e}", f"{res['lambda_eff_n']:.2e}", f"{res['tau_phi_inv']:.2e}", f"{res['F_ltz']:.2e}",
              f"{res['F_ltz_j']:.2e}", f"{res['F_ltz_p']:.2e}"]
    print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(name, *values))

print()
print()

# --- Table 4 ---

header = ['Target', 'ρ_B', 'ρ_B_p', 'G_EM', 'M_decoup', 'M_j_outf', 'E²', 'p_rel', 'B_grav']
print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(*header))
for name, res in results.items():
    params = targets[name]
    values = [f"{res['rho_B']:.2e}", f"{res['rho_B_p']:.2e}",
              f"{res['G_EM']:.2e}", f"{res['M_decoup']:.2e}", f"{res['M_j_outf']:.2e}", f"{res['E_squared']:.2e}",
              f"{res['p_rel']:.2e}", f"{res['B_grav']:.2e}"]
    print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(name, *values))

print()
print()

# --- Table 5 ---

header = ['Target', 'g',
          'dP_dt', 'ρ_comp', 'R_bubble_t', 'div_E', 'curl_E', 'curl_B', 'F_unified']
print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(*header))
for name, res in results.items():
    params = targets[name]
    values = [f"{res['g']:.2e}", f"{res['dP_dt']:.2e}",
              f"{res['rho_compressed']:.2e}", f"{res['R_bubble_t']:.2e}", f"{res['div_E']:.2e}", f"{res['curl_E']:.2e}",
              f"{res['curl_B']:.2e}", f"{res['F_unified']:.2e}"]
    print("{:<20} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12} {:<12}".format(name, *values))




Processing Sun...
Processing Alpha Centauri A...
Processing Sirius B...
Processing Tau Ceti...
Processing Delta Pavonis...
Processing Beta Hydri...
Processing Crab Pulsar...
Processing 1E 1841-045...
Processing SGR 1806-20...
Processing TON 618...

Final Complete Validation Results:
Target               Type           Distance (ly)   Avg Φ        Avg E_Φ      G_eff        P_anom       λ_eff_27     σ_g         
Sun                  Main-Sequence  0.00            2.64e-04     -5.93e-10    6.68e-11     1.01e-16     1.13e-03     4.00e-01    
Alpha Centauri A     Main-Sequence  4.37            2.44e-04     -5.08e-10    6.68e-11     4.17e-16     1.13e-03     4.00e-01    
Sirius B             White Dwarf    8.60            3.25e+00     -8.85e+03    1.06e-05     1.59e-09     1.13e-03     4.00e-01    
Tau Ceti             Main-Sequence  11.90           3.26e-04     -9.04e-10    6.68e-11     2.46e-17     1.13e-03     4.00e-01    
Delta Pavonis        Subgiant       19.90           2.28e-04     -